In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
test_data = pd.read_csv("./Calibration_Data_1.csv")

# Constant Offset

In [ ]:
test_data.plot()

## simple offset


In [ ]:
simple_offset = test_data["actual"].mean() - test_data["device"].mean()

In [ ]:
simple_offset_data = test_data.copy(True)

In [ ]:
simple_offset_data["offset"] = simple_offset_data["device"] + simple_offset

In [ ]:
simple_offset_data.plot()

In [ ]:
simple_offset_data["difference"] = simple_offset_data["actual"]-simple_offset_data["offset"]

In [ ]:
simple_offset_std = simple_offset_data["difference"].std()
print(simple_offset_std)

# Polynomial Regression

In [ ]:
coef = []
max_order = 21

In [ ]:
poly_data = test_data.copy(True)

In [ ]:
for i in range(1, max_order):
    coef.append(np.polyfit(test_data["device"], test_data["actual"], deg=i))

    poly_data["order "+str(i)] = np.polyval(coef[-1], test_data["device"])

In [ ]:
poly_data.plot(ylim=(36.1, 37.2))

In [ ]:
pl = ["actual"]
for i in [4]:
    pl.append("order "+str(i))
poly_data.plot(y=pl)

In [ ]:
poly_stds = []

In [ ]:
for i in range(1,max_order):
    poly_data["difference "+str(i)] = poly_data["actual"] - poly_data["order "+str(i)]
    poly_stds.append(poly_data["difference "+str(i)].std())

In [ ]:
poly_stds

Order 3 seems to give the most accurate approcimation without over fitting.


In [ ]:
poly_std = poly_stds[2]

# Kalman filter


# Overview

In [ ]:

overview = pd.DataFrame({
    "Methed" : ["Constant Offset", "Ploynomial Regression"],
    "Standard deviation" : [simple_offset_std, poly_std],
    "Parameters": [simple_offset, [round(v, 5) for v in coef[1]]]
})

In [ ]:
overview

In [ ]:
x = np.arange(min(test_data["device"])-1, max(test_data["device"]+1), 0.01)

In [ ]:
plt.scatter(test_data["device"], test_data["actual"], label="Actual vs Device")
plt.plot(x, np.polyval(coef[0], x), label="Poly Order 1")
plt.plot(x, np.polyval(coef[1], x), label="Poly Order 2")
plt.plot(x, np.polyval(coef[2], x), label="Poly Order 3")
# plt.plot(x, x+simple_offset, label="Simple Offset")
plt.plot(x, np.polyval(coef[3], x), label="Poly Order 4")
plt.xlabel("Device")
plt.ylabel("Actual")
plt.title("Actual vs Device Data")
plt.legend()
plt.show()

In [ ]:
rows = math.ceil(max_order/2)
fig, axs = plt.subplots(rows, 2, figsize=(14, 6*rows))

# Simple Offset
axs[0, 0].plot(simple_offset_data.index, simple_offset_data["actual"], label="Actual")
axs[0, 0].plot(simple_offset_data.index, simple_offset_data["offset"], label="Simple Offset")
axs[0, 0].set_title(f"Simple Offset\nStd: {simple_offset_std:.3f}")
axs[0, 0].set_xlabel("Time")
axs[0, 0].set_ylabel("Temperature")
axs[0, 0].legend()

# Polynomial Orders 1-5
for i in range(1, max_order):
    row = i // 2
    col = i % 2
    ax = axs[row, col]
    ax.plot(poly_data.index, poly_data["actual"], label="Actual")
    ax.plot(poly_data.index, poly_data[f"order {i}"], label=f"Poly Order {i}")
    std = poly_stds[i-1]
    ax.set_title(f"Polynomial Order {i}\nStd: {std:.6f}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Temperature")
    ax.legend()

# Hide any unused subplot (in this case, axs[2, 1])
if len(axs.flatten()) > 6:
    for j in range(6, len(axs.flatten())):
        axs.flatten()[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(rows, 2, figsize=(14, 6*rows))

# Simple Offset
axs[0, 0].scatter(test_data["device"], test_data["actual"], label="Actual vs Device")
axs[0, 0].plot(x, x+simple_offset, label="Simple Offset")
axs[0, 0].set_title(f"Simple Offset\nStd: {simple_offset_std:.3f}")
axs[0, 0].set_xlabel("Device Temperature")
axs[0, 0].set_ylabel("Actual Temperature")
axs[0, 0].legend()

# Polynomial Orders 1-5
for i in range(1, max_order):
    row = i // 2
    col = i % 2
    ax = axs[row, col]
    ax.scatter(test_data["device"], test_data["actual"], label="Actual vs Device")
    ax.plot(x, np.polyval(coef[i-1], x), label="Poly Order "+str(i))
    std = poly_stds[i-1]
    ax.set_title(f"Polynomial Order {i}\nStd: {std:.6f}")
    ax.set_xlabel("Device Temperature")
    ax.set_ylabel("Actual Temperature")
    ax.legend()

# Hide any unused subplot (in this case, axs[2, 1])
if len(axs.flatten()) > 6:
    for j in range(6, len(axs.flatten())):
        axs.flatten()[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
poly_stds

In [ ]:
test = np.arange(0, max_order, 1)

plt.plot(test[1:], poly_stds, label="Test Line")
plt.xlabel("Polynomial Order")
plt.ylabel("Standard Deviation")
plt.title("Standard Deviation of Polynomial Orders")
plt.xticks(test[1:])
plt.grid()

In [ ]:
orders = [ 3,5]
if max(orders) > max_order:
    raise ValueError(f"Maximum order {max(orders)} exceeds defined max_order {max_order}.")
rows = math.ceil(len(orders)/2)
if rows <2:
    rows = 2  # Ensure at least one row for plotting
fig, axs = plt.subplots(rows, 2, figsize=(14, 6*rows))


# Polynomial Orders 1-5
for i, v in enumerate(orders, start=0):
    row = i // 2
    col = i % 2
    ax = axs[row, col]
    ax.plot(poly_data.index, poly_data["actual"], label="Actual")
    ax.plot(poly_data.index, poly_data[f"order {v}"], label=f"Poly Order {v}")
    std = poly_stds[v-1]
    ax.set_title(f"Polynomial Order {v}\nStd: {std:.6f}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Temperature")
    ax.legend()

# Hide any unused subplot (in this case, axs[2, 1])
if len(axs.flatten()) > 6:
    for j in range(6, len(axs.flatten())):
        axs.flatten()[j].axis('off')

plt.tight_layout()
plt.show()